# Combined Indicators

This notebook is meant to compile all of the indicators into a single dataframe for easy analysis.  

The steps required to do this are:  
* Gather data from source web sites  
* Clean data and choose important data points to put into a single .csv or database  
* Eliminate trends from data using Mayer's multiple and/or z-scores  

Table of Contents:  
* Quarterly GDP Data  
* Monthly Chicago Fed National Activity Index  
* Monthly CPI Data  
* Monthly Consumer Sentiment Data

### Quarterly GDP Data

This data requires use of EMAs and Mayer's multiple to eliminate trend effects.

In [108]:
import pandas as pd

df_gdp = pd.read_csv('../0_inputs/economic_indicators/bea_116_real_gdp_cleaned.csv')
df_gdp.tail(3)

,year,quarter,gross_domestic_product,personal_consumption_expenditures,goods,durable_goods,nondurable_goods,services,gross_private_domestic_investment,fixed_investment,...,services.1,imports,goods1,services1,government_consumption_expenditures_and_gross_investment,federal,national_defense,nondefense,state_and_local,residual
298,2021,Q3,19478.9,13732.4,5518.3,2158.5,3394,8378.5,3609.7,3585,...,599.8,3589.6,3122.8,492.2,3381.6,1339.1,794.3,543.9,2042.1,-57.1
299,2021,Q4,19806.3,13818.4,5533.9,2171.9,3397.1,8446.2,3903.0,3609.1,...,627.7,3740.8,3261.1,507.6,3359.0,1324.3,782,541.2,2034,-44.4
300,2022,Q1,19735.9,13911.5,5532.3,2194,3375.9,8534.9,3925.3,3673.2,...,633.6,3896.3,3416.7,512.7,3335.8,1304.3,764.8,538.2,2030.1,-19.5


In [109]:
# This data contains --- where there are blanks
df_gdp = df_gdp.replace(to_replace='---',value=0)

In [ ]:
# Interquartile Range Code to eliminate outliers
# quartiles = df_gdp2['_mayer'].quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq
# iqr_1_5 = iqr * 1.5
# lower_bound = lowerq - iqr_1_5
# upper_bound = upperq + iqr_1_5

In [110]:
cols = list(df_gdp.columns)
col_zscore_list = ['year','quarter']
col_zscore_chg_list = []

for col in cols:
    col_5yr = col + '_5yr'
    col_mayer = col + '_mayer'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        df_gdp[col_5yr] = df_gdp[col].ewm(span=20,adjust=False).mean()
        df_gdp[col_mayer] = (df_gdp[col] - df_gdp[col_5yr]) / df_gdp[col_5yr] * 100
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')

# Eliminate first 20 rows of data
df_gdp2 = df_gdp[20:]

cols = list(df_gdp2.columns)

for col in cols:
    col_mayer = col + '_mayer'
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        df_gdp2[col_zscore] = (df_gdp2[col_mayer] - df_gdp2[col_mayer].mean())/df_gdp2[col_mayer].std(ddof=0)
        df_gdp2[col_chg] = df_gdp2[col_mayer].diff()
        df_gdp2[col_chg_zscore] = (df_gdp2[col_chg] - df_gdp2[col_chg].mean())/df_gdp2[col_chg].std(ddof=0)
    except:
        print(f'No numeric types in {col}.')
        
df2 = df_gdp2[col_zscore_list+col_zscore_chg_list]
df2.drop(['year_zscore','year_chg_zscore'],axis=1,inplace=True)
df2.to_csv('./Output/Quarterly_Data/GDP_116.csv')
df2.tail(5)

No numeric types in quarter.
No numeric types in goods.
No numeric types in durable_goods.
No numeric types in nondurable_goods.
No numeric types in services.
No numeric types in fixed_investment.
No numeric types in nonresidential.
No numeric types in structures.
No numeric types in equipment.
No numeric types in intellectual_property_products.
No numeric types in residential.
No numeric types in change_in_private_inventories.
No numeric types in net_exports_of_goods_and_services.
No numeric types in goods.1.
No numeric types in services.1.
No numeric types in goods1.
No numeric types in services1.
No numeric types in federal.
No numeric types in national_defense.
No numeric types in nondefense.
No numeric types in state_and_local.
No numeric types in quarter.
No numeric types in goods.
No numeric types in durable_goods.
No numeric types in nondurable_goods.
No numeric types in services.
No numeric types in fixed_investment.
No numeric types in nonresidential.
No numeric types in stru

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,year,quarter,gross_domestic_product_zscore,personal_consumption_expenditures_zscore,gross_private_domestic_investment_zscore,exports_zscore,imports_zscore,government_consumption_expenditures_and_gross_investment_zscore,residual_zscore,gross_domestic_product_chg_zscore,personal_consumption_expenditures_chg_zscore,gross_private_domestic_investment_chg_zscore,exports_chg_zscore,imports_chg_zscore,government_consumption_expenditures_and_gross_investment_chg_zscore,residual_chg_zscore
296,2021,Q1,-1.184664,-0.942987,-0.202502,-2.043672,-0.819065,-0.264459,0.173402,1.079342,2.205351,-0.356430,-0.056140,0.436092,0.703876,-0.043966
297,2021,Q2,-0.764429,-0.074523,-0.404416,-1.785642,-0.689532,-0.410876,0.164791,1.075490,2.150175,-0.415151,0.530520,0.261518,-0.635334,-0.005732
298,2021,Q3,-0.751567,-0.183255,-0.131302,-1.908862,-0.647682,-0.424347,0.043970,0.052039,-0.275819,0.540237,-0.242068,0.084238,0.052300,-0.086694
299,2021,Q4,-0.356906,-0.228820,0.713943,-1.291313,-0.210186,-0.580266,0.011893,1.011239,-0.119000,1.690921,1.259073,0.884166,-0.684481,-0.022663
300,2022,Q1,-0.708174,-0.251808,0.593677,-1.484277,0.183875,-0.723472,-0.049509,-0.862770,-0.062949,-0.250938,-0.383403,0.796347,-0.618728,-0.043822


In [91]:
# Code used to test accuracy of ema calculation
# import yfinance as yf

# ticker = yf.Ticker('SPY')
# # Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
# ticker_hist = ticker.history(period="5y")
# ticker_hist2 = ticker_hist.reset_index()
# df = ticker_hist2

# df['ema_20'] = df['Close'].ewm(span=20,adjust=False).mean()
# df[['ema_20']]

In [92]:
# df_T['EMA'] = df_T.iloc[:,0].ewm(span=40,adjust=False).mean()

### Chicago Fed National Activity Index (CFNAI)

In [6]:
import pandas as pd

df_cfnai = pd.read_csv('../0_inputs/economic_indicators/cfnai-data-series-xlsx_cleaned.csv')

# Eliminate first 2 rows for 3 month MA data
df_cfnai = df_cfnai[2:]
df_cfnai.tail(3)

,year,month,prod_and_income,employment,consumption_and_housing,sales_orders_inventories,cfnai,cfnai_ma3,diffusion
658,2022,1,0.17,0.13,0.28,0.16,0.75,0.44,0.33
659,2022,2,0.30,0.31,-0.04,-0.03,0.54,0.43,0.31
660,2022,3,0.27,0.16,-0.04,0.06,0.44,0.57,0.45


In [7]:
cols = list(df_cfnai.columns)
col_zscore_list = ['year','month']
col_zscore_chg_list = []

for col in cols:
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
    try:
        df_cfnai[col_zscore] = (df_cfnai[col] - df_cfnai[col].mean())/df_cfnai[col].std(ddof=0)
        df_cfnai[col_chg] = df_cfnai[col].diff()
        df_cfnai[col_chg_zscore] = (df_cfnai[col_chg] - df_cfnai[col_chg].mean())/df_cfnai[col_chg].std(ddof=0)
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')
        
df_cfnai2 = df_cfnai[col_zscore_list+col_zscore_chg_list]
df_cfnai2.drop(['year_zscore','year_chg_zscore','month_zscore','month_chg_zscore'],axis=1,inplace=True)
df_cfnai2.to_csv('./Output/Monthly_Data/chicago_fed_national_activity_index_output.csv')
df_cfnai2.tail(5)

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,year,month,prod_and_income_zscore,employment_zscore,consumption_and_housing_zscore,sales_orders_inventories_zscore,cfnai_zscore,cfnai_ma3_zscore,diffusion_zscore,prod_and_income_chg_zscore,employment_chg_zscore,consumption_and_housing_chg_zscore,sales_orders_inventories_chg_zscore,cfnai_chg_zscore,cfnai_ma3_chg_zscore,diffusion_chg_zscore
656,2021,11,0.601917,0.693222,-0.078586,0.238230,0.559163,0.646589,1.048463,-0.446871,0.056423,-0.473331,-0.10789,-0.207278,0.399129,1.055420
657,2021,12,-0.341459,0.618880,-1.398525,0.381124,0.009553,0.675961,1.048463,-0.802477,-0.057287,-1.149371,0.10637,-0.454509,0.041722,-0.006557
658,2022,1,0.444687,0.321511,2.173075,0.762176,0.749029,0.646589,0.730839,0.664396,-0.227853,3.109681,0.28492,0.608583,-0.047629,-0.803039
659,2022,2,0.785351,0.767565,-0.311516,-0.142822,0.539178,0.631903,0.677902,0.286565,0.340700,-2.163431,-0.67925,-0.174314,-0.025291,-0.139304
660,2022,3,0.706736,0.395853,-0.311516,0.285861,0.439248,0.837508,1.048463,-0.069040,-0.284708,-0.000103,0.32063,-0.083663,0.309778,0.922673


### Monthly CPI
This data requires use of Mayer's to eliminate trend effects.

In [16]:
import pandas as pd

cpi_df = pd.read_csv('./Inputs/cpi_compiled.csv')
cpi_df.drop('Unnamed: 0',axis=1,inplace=True)
cpi_df.tail(3)

,id,year,month,commodities,cpi,day_care,durables,education,energy,food,housing,meat,medical_care,motor_fuel,new_vehicles,used_vehicles
1308,202201.0,2022.0,1,210.918,281.148,323.184,127.345,278.087,260.653,288.264,289.889,298.441,535.048,291.767,167.582,210.293
1309,202202.0,2022.0,2,213.960,283.716,324.504,128.109,278.380,267.771,291.244,291.504,301.240,536.932,307.422,168.027,212.040
1310,202203.0,2022.0,3,219.057,287.504,324.205,127.471,278.388,298.246,294.064,293.577,305.116,539.739,368.440,168.056,208.216


In [17]:
cols = list(cpi_df.columns)
col_mayer_list = []
col_zscore_list = ['id','year','month']
col_zscore_chg_list = []

for col in cols:
    col_5yr = col + '_5yr'
    col_mayer = col + '_mayer'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        cpi_df[col_5yr] = cpi_df[col].ewm(span=60,adjust=False).mean()
        cpi_df[col_mayer] = (cpi_df[col] - cpi_df[col_5yr]) / cpi_df[col_5yr] * 100
        col_mayer_list.append(col + '_mayer')
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')

# Eliminate first 20 rows of data
cpi_df2 = cpi_df[col_mayer_list]
cpi_df2 = cpi_df[60:]

cols = list(cpi_df2.columns)

for col in cols:
    col_mayer = col + '_mayer'
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        cpi_df2[col_zscore] = (cpi_df2[col_mayer] - cpi_df2[col_mayer].mean())/cpi_df2[col_mayer].std(ddof=0)
        cpi_df2[col_chg] = cpi_df2[col_mayer].diff()
        cpi_df2[col_chg_zscore] = (cpi_df2[col_chg] - cpi_df2[col_chg].mean())/cpi_df2[col_chg].std(ddof=0)
    except:
        print(f'No numeric types in {col}.')
        
cpi_df3 = cpi_df2[col_zscore_list+col_zscore_chg_list]
cpi_df3.drop(['year_zscore','year_chg_zscore',
             'id_zscore','id_chg_zscore',
             'month_zscore','month_chg_zscore'],axis=1,inplace=True)
cpi_df3.to_csv('./Output/Monthly_Data/cpi.csv')
cpi_df3.tail(5)

No numeric types in id_5yr.
No numeric types in id_mayer.
No numeric types in year_5yr.
No numeric types in year_mayer.
No numeric types in month_5yr.
No numeric types in month_mayer.
No numeric types in commodities_5yr.
No numeric types in commodities_mayer.
No numeric types in cpi_5yr.
No numeric types in cpi_mayer.
No numeric types in day_care_5yr.
No numeric types in day_care_mayer.
No numeric types in durables_5yr.
No numeric types in durables_mayer.
No numeric types in education_5yr.
No numeric types in education_mayer.
No numeric types in energy_5yr.
No numeric types in energy_mayer.
No numeric types in food_5yr.
No numeric types in food_mayer.
No numeric types in housing_5yr.
No numeric types in housing_mayer.
No numeric types in meat_5yr.
No numeric types in meat_mayer.
No numeric types in medical_care_5yr.
No numeric types in medical_care_mayer.
No numeric types in motor_fuel_5yr.
No numeric types in motor_fuel_mayer.
No numeric types in new_vehicles_5yr.
No numeric types in 

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,id,year,month,commodities_zscore,cpi_zscore,day_care_zscore,durables_zscore,education_zscore,energy_zscore,food_zscore,...,durables_chg_zscore,education_chg_zscore,energy_chg_zscore,food_chg_zscore,housing_chg_zscore,meat_chg_zscore,medical_care_chg_zscore,motor_fuel_chg_zscore,new_vehicles_chg_zscore,used_vehicles_chg_zscore
1306,202111.0,2021.0,11,0.216441,0.100530,-0.956149,0.827689,-1.409093,0.812833,-0.129969,...,2.188315,-0.320509,0.358623,0.301665,0.223261,0.094139,0.215525,0.525184,0.809585,1.130346
1307,202112.0,2021.0,12,0.227823,0.104422,-1.026045,0.983064,-1.485732,0.652994,-0.089086,...,3.073839,-0.455610,-0.796173,0.410692,0.424673,-0.762620,0.105350,-0.854059,0.858836,1.759686
1308,202201.0,2022.0,1,0.329321,0.180003,-1.045277,1.082826,-1.519622,0.755714,0.042439,...,1.970467,-0.214431,0.484797,1.395692,1.627479,-0.130400,2.507305,-0.201858,0.175035,0.267780
1309,202202.0,2022.0,2,0.472305,0.262987,-0.962450,1.092997,-1.540656,0.944904,0.187221,...,0.192974,-0.141907,0.906673,1.539751,0.763921,0.349433,0.464351,1.213698,-0.089961,-0.248994
1310,202203.0,2022.0,3,0.738478,0.400728,-1.092194,0.961013,-1.591155,1.918373,0.314443,...,-2.627379,-0.308132,4.733025,1.348925,1.201824,0.591098,1.016335,5.193383,-0.306509,-2.193690


### Consumer Sentiment

In [5]:
import pandas as pd

df_cons_sent = pd.read_csv('./Inputs/Consumer_Sentiment/cons_sentiment_cleaned.csv')
df_cons_sent.drop('Unnamed: 0',axis=1,inplace=True)
df_cons_sent.tail(3)

,id,year,month,cons_sent
621,202202,2022,2,62.8
622,202203,2022,3,59.4
623,202204,2022,4,65.2


In [6]:
cols = list(df_cons_sent.columns)
col_zscore_list = ['id','year','month']
col_zscore_chg_list = []

for col in cols:
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
    try:
        df_cons_sent[col_zscore] = (df_cons_sent[col] - df_cons_sent[col].mean())/df_cons_sent[col].std(ddof=0)
        df_cons_sent[col_chg] = df_cons_sent[col].diff()
        df_cons_sent[col_chg_zscore] = (df_cons_sent[col_chg] - df_cons_sent[col_chg].mean())/df_cons_sent[col_chg].std(ddof=0)
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')
        
df_cons_sent2 = df_cons_sent[col_zscore_list+col_zscore_chg_list]
df_cons_sent2.drop(['year_zscore','year_chg_zscore',
                    'month_zscore','month_chg_zscore',
                   'id_zscore','id_chg_zscore'],axis=1,inplace=True)
df_cons_sent2.to_csv('./Output/Monthly_Data/cons_sentiment_data.csv')
df_cons_sent2.tail(5)

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\pandas\core\frame.py:4908: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,id,year,month,cons_sent_zscore,cons_sent_chg_zscore
619,202112,2021,12,-1.265542,0.767385
620,202201,2022,1,-1.540582,-0.798848
621,202202,2022,2,-1.896517,-1.036156
622,202203,2022,3,-2.171558,-0.798848
623,202204,2022,4,-1.702371,1.384386


### PPI Data

In [14]:
import pandas as pd

ppi_df = pd.read_csv('./Inputs/ppi_compiled.csv')
ppi_df.drop('Unnamed: 0',axis=1,inplace=True)
ppi_df.tail(3)

,id,year,month,copper,farm_products,farm_products_plus,fuels_and_power,furniture,industrial_commodities,lumber,machinery,metals,ppi_all,processed_foods_and_feeds,rubber_and_plastics,warehousing
1308,202201.0,2022.0,1,548.592,220.224,234.399,226.568,201.306,248.133,348.380,157.958,333.297,246.245,241.662,241.326,127.634
1309,202202.0,2022.0,2,552.152,230.899,241.037,248.504,203.182,255.120,359.649,159.468,328.447,253.180,246.368,243.541,122.137
1310,202203.0,2022.0,3,572.186,246.991,248.932,259.635,204.895,260.496,372.099,160.735,334.957,258.999,250.327,246.072,122.081


In [15]:
cols = list(ppi_df.columns)
col_mayer_list = []
col_zscore_list = ['id','year','month']
col_zscore_chg_list = []

for col in cols:
    col_5yr = col + '_5yr'
    col_mayer = col + '_mayer'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        ppi_df[col_5yr] = ppi_df[col].ewm(span=60,adjust=False).mean()
        ppi_df[col_mayer] = (ppi_df[col] - ppi_df[col_5yr]) / ppi_df[col_5yr] * 100
        col_mayer_list.append(col + '_mayer')
        col_zscore_list.append(col + '_zscore')
        col_zscore_chg_list.append(col + '_chg_zscore')
    except:
        print(f'No numeric types in {col}.')

# Eliminate first 5 years of data
# ppi_df2 = ppi_df[col_mayer_list]
ppi_df2 = ppi_df[60:]

cols = list(ppi_df2.columns)

for col in cols:
    col_mayer = col + '_mayer'
    col_zscore = col + '_zscore'
    col_chg = col + '_chg'
    col_chg_zscore = col + '_chg_zscore'
#     df_gdp[col_5yr] = df_gdp[col].rolling(20)
#     df_gdp[col_5yr] = df_gdp.iloc[:,0].ewm(span=20,adjust=False).mean()
    try:
        ppi_df2[col_zscore] = (ppi_df2[col_mayer] - ppi_df2[col_mayer].mean())/ppi_df2[col_mayer].std(ddof=0)
        ppi_df2[col_chg] = ppi_df2[col_mayer].diff()
        ppi_df2[col_chg_zscore] = (ppi_df2[col_chg] - ppi_df2[col_chg].mean())/ppi_df2[col_chg].std(ddof=0)
    except:
        print(f'No numeric types in {col}.')
        
ppi_df3 = ppi_df2[col_zscore_list+col_zscore_chg_list]
ppi_df3.drop(['year_zscore','year_chg_zscore',
             'id_zscore','id_chg_zscore',
             'month_zscore','month_chg_zscore'],axis=1,inplace=True)
ppi_df3.to_csv('./Output/Monthly_Data/ppi.csv')
ppi_df3.tail(5)

No numeric types in id_5yr.
No numeric types in id_mayer.
No numeric types in year_5yr.
No numeric types in year_mayer.
No numeric types in month_5yr.
No numeric types in month_mayer.
No numeric types in copper_5yr.
No numeric types in copper_mayer.
No numeric types in farm_products_5yr.
No numeric types in farm_products_mayer.
No numeric types in farm_products_plus_5yr.
No numeric types in farm_products_plus_mayer.
No numeric types in fuels_and_power_5yr.
No numeric types in fuels_and_power_mayer.
No numeric types in furniture_5yr.
No numeric types in furniture_mayer.
No numeric types in industrial_commodities_5yr.
No numeric types in industrial_commodities_mayer.
No numeric types in lumber_5yr.
No numeric types in lumber_mayer.
No numeric types in machinery_5yr.
No numeric types in machinery_mayer.
No numeric types in metals_5yr.
No numeric types in metals_mayer.
No numeric types in ppi_all_5yr.
No numeric types in ppi_all_mayer.
No numeric types in processed_foods_and_feeds_5yr.
No 

C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\srobi\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,id,year,month,copper_zscore,farm_products_zscore,farm_products_plus_zscore,fuels_and_power_zscore,furniture_zscore,industrial_commodities_zscore,lumber_zscore,...,fuels_and_power_chg_zscore,furniture_chg_zscore,industrial_commodities_chg_zscore,lumber_chg_zscore,machinery_chg_zscore,metals_chg_zscore,ppi_all_chg_zscore,processed_foods_and_feeds_chg_zscore,rubber_and_plastics_chg_zscore,warehousing_chg_zscore
1306,202111.0,2021.0,11,1.020400,0.968945,0.962801,1.472236,1.137501,1.311058,0.520129,...,0.046513,0.407610,0.481764,0.494588,1.009034,1.194361,0.522096,-0.244977,0.827911,-1.699940
1307,202112.0,2021.0,12,0.756724,1.197764,1.054026,0.956642,1.182169,1.102028,0.992210,...,-2.833857,0.480097,-1.727742,2.972714,1.190582,-1.624215,-1.270039,-0.292635,0.629544,3.276339
1308,202201.0,2022.0,1,0.884746,1.383875,1.287184,1.220568,1.311574,1.264022,1.773424,...,1.432268,1.433713,1.348035,4.929843,2.850970,-0.988349,1.312105,1.001317,0.682929,6.032797
1309,202202.0,2022.0,2,0.868257,1.706002,1.627414,1.861948,1.410397,1.510729,1.993069,...,3.497995,1.089558,2.050314,1.374547,1.543901,-3.078627,1.975532,1.244327,0.215580,-5.705158
1310,202203.0,2022.0,3,1.029615,2.202399,2.028040,2.113002,1.489066,1.670415,2.230431,...,1.361825,0.862733,1.328899,1.486710,1.102170,0.958029,1.463015,0.936448,0.313715,-0.577647
